<a href="https://colab.research.google.com/github/profteachkids/CHE3022/blob/main/NetworkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import newton, root

In [2]:
g=9.81
rho=1e3
mu=1e-3
D=0.1
eD=1e-3

In [3]:
# Calculates the friction factor using Churchill correlation 
def f(Re,eD):
    A=(2.457*np.log(1/((7./Re)**0.9+0.27*eD)))**16
    B=(37530/Re)
    return 2*((8/Re)**12 + 1/(A+B)**(1.5))**(1/12)

In [4]:
# Given a flow Q, calculates delta Head  (head at ToNode) - (head at FromNode)
def dHead(Q,L,K,D,eD):
    v=np.abs(Q/(np.pi*D**2/4))  #ensures that we do not end up with negative Reynolds numbers
    Re= rho*v*D/mu
    return -np.sign(Q)*(K+4*f(Re,eD)*L/D)*(v**2/2)/g  #incorporate back in sign (directionality) of Q (flow)

In [5]:
# Given deltaHead, it calculates the flow (with proper signage)
def flow(deltaHead, L, K, D, eD):
    dH_abs=np.abs(deltaHead)  #ensures no negative Re
    if dH_abs<1e-10: #takes care of situation where deltaHead is extremely small, for which
    # the numerical solver (newton) has difficulty converging
        return 0.0

    Qguess=np.pi*D**2/4*np.sqrt(dH_abs/(K+4*0.01*L/D)*2*g)  #Initial guess for flow rate, assuming a reasonable
    #guess of 0.01 for the Fanning friction factor

    #function to zero - Calculated deltahead based on the iterated flow rate (Q) must be equal to the deltaHead
    #given.
    def f_to_zero(Q):
        return dHead(Q,L,K,D,eD)-dH_abs

    resulting_flow_rate=newton(f_to_zero,Qguess)

    return np.sign(deltaHead)*resulting_flow_rate  #Correct for directionality of deltaHead

In [14]:
#fromNode, toNode, L, K
pipes = np.array([[0, 1, 1.5, 1.5],
                  [0, 2, 2., 2.],
                  [1,2, 1.5, 0],
                  [1, 3, 1.5, 0],
                  [2,4, 0.5, 0],
                  [3,4, 3., 2]])

z=np.array([30.,0,0,0,0 ])
externalP=np.array([2e5, 0.])
externalNodes=[0, 4]
fromNodes=(pipes[:,0]).astype(np.int32)
toNodes=(pipes[:,1]).astype(np.int32)
lengths=pipes[:,2]
K = pipes[:,3]

nPipes = pipes.shape[0]
pipeD = np.full(nPipes, D)
pipe_eD = np.full(nPipes,eD)
nNodes = (np.max(pipes[:,1])).astype(np.int32)+1
heads=np.zeros(nNodes)
heads[externalNodes]=externalP/rho/g + z[externalNodes]

In [7]:
internalNodes=list(set(range(nNodes))-set(externalNodes))

In [8]:
def calc_flows(heads):
    flows=np.zeros(nPipes)
    for i in range(nPipes):
        deltaHead = heads[toNodes[i]] - heads[fromNodes[i]]
        flows[i]=flow(deltaHead, lengths[i], K[i], pipeD[i], pipe_eD[i])
    return flows

def calc_netQ(flows):
    netQ=np.zeros(nNodes)
    np.add.at(netQ,toNodes,flows)
    np.add.at(netQ,fromNodes,-flows)
    return netQ

In [9]:
calc_netQ(calc_flows(heads))

array([-0.3437157 ,  0.18420574,  0.15950995,  0.        ,  0.        ])

In [10]:
def netQ_internal(internalP):
    heads[internalNodes]=internalP/rho/g + z[internalNodes]
    return calc_netQ(calc_flows(heads))[internalNodes]


In [11]:
internalHeadGuess=np.mean(heads[externalNodes])
internalPGuess = (internalHeadGuess - z[internalNodes])*rho*g 

In [16]:
result=root(netQ_internal, internalPGuess)

In [17]:
result.x

array([82620.70449207, 53908.25550016, 74034.72098594])

In [20]:
calc_flows(heads)

array([0.16809655, 0.15055386, 0.10887172, 0.05922483, 0.25942558,
       0.05922483])

In [21]:
calc_netQ(calc_flows(heads))

array([-3.18650410e-01,  2.95076463e-13,  1.58761893e-14, -3.01141057e-13,
        3.18650410e-01])